# Scraping Twitter using Twitter API

## Introduction

Here, we are implementing an easy-to-use Python library called Tweepy for accessing the Twitter API. Through Twitter API, we will extract recent tweets related to the stock market along with their users. The respective tweet and user details are further stored in a dataframe as per the requirement. Finally, data from each of these dataframes is inserted into the MySQL database. 

### Importing Essential Libraries

In [1]:
import tweepy #Library required for accessing the Twitter API
import pandas as pd
import mysql.connector #Library required for connecting to the MySQL Database

### Authenticating Keys

Here, we are defining keys to authenticate with the Twitter API and start calling API functions to extract tweets for our analysis.
#### Kindly enter your own keys to run this code.

In [2]:
api_key = "XXXXXXXXXXXXXXXXXXXXXXXXXXXX"
api_key_secret = "XXXXXXXXXXXXXXXXXXXXXXXXXXXX"
access_token = "XXXXXXXXXXXXXXXXXXXXXXXXXXXX"
access_token_secret = "XXXXXXXXXXXXXXXXXXXXXXXXXXXX"

### Connecting to the MySQL Database

In [3]:
mydb = mysql.connector.connect(host='localhost', user = 'root', passwd = 'Northeastern@NEU10', database = 'stock_assist')
mycursor = mydb.cursor()

### OAuth Authentication
Twitter API requires all the requests to be authenticated via OAuth. That is why, we need to create the following credentials to be able to access the Twitter API:
- API Key
- API Key Secret
- Access Token
- Access Token Secret

In [4]:
authenticator = tweepy.OAuthHandler(api_key, api_key_secret)
authenticator.set_access_token(access_token, access_token_secret)
api = tweepy.API(authenticator, wait_on_rate_limit = True)

In [5]:
#Initialize lists to hold the Twitter details
outtweets = [] #initialize master list to hold our ready tweets
tbl_users = []
tbl_tweet = []
tbl_tags = []
tbl_mention = []
hashtag_list = []
mention_list = []

In [6]:
#Initialize list of the keywords based on which tweets will be fetched
keywords = ['#stockmarket', '#stocks', '#finance', '#AAPL', '#AMZN']

## Fetching Twitter data based on the keyword

In [7]:
for i in keywords:
    for tweet in tweepy.Cursor(api.search_tweets, q=i, count= 100, lang="en", since_id = '2022-11-10').items(50):
        print(tweet)
        media = tweet.entities.get('media', [])
        if tweet.entities.get('media', []):
            #Fetching user data
            name = tweet.user.screen_name
            user = api.get_user(screen_name=name)
            status = api.get_status(tweet.id)

            #Splitting the text into words
            for word in tweet.text.split():

                #Checking the first character of every word
                if word[0] == '#':
                    #Adding the word to the hashtag_list
                    hashtag_list.append(word[1:])
            #Combine all the tags from the hashtag_list into a single attribute
            tags = ','.join(hashtag_list)

            #Splitting the text into words
            for word in tweet.text.split():

                    #Checking the first character of every word
                if word[0] == '@' and len(word) > 1:
                    #Adding the word to the hashtag_list
                    mention_list.append(word[1:])
            #Combine all the tags from the mention_list into a single attribute
            mentions = ','.join(mention_list)

            #Append the Twitter user and tweet data into their respective lists
            outtweets.append([tweet.id, tweet.user.screen_name, tweet.user.name, tweet.user.profile_image_url, tweet.user.description, user.followers_count, user.friends_count, user.created_at, tweet.text, tags, tweet.created_at, mentions, tweet.retweet_count])
            tbl_users.append([tweet.user.screen_name, tweet.user.name, tweet.user.profile_image_url, tweet.user.description, user.followers_count, user.friends_count, user.created_at])
            tbl_tweet.append([tweet.id, tweet.user.screen_name, tweet.text, tweet.user.profile_image_url,tweet.created_at, tweet.retweet_count])
            
            for i in hashtag_list:
                tbl_tags.append([tweet.id, i])

            for j in mention_list:
                tbl_mention.append([tweet.id, tweet.user.screen_name, j])


Status(_api=<tweepy.api.API object at 0x000002AF01C19D88>, _json={'created_at': 'Sat Nov 12 21:02:24 +0000 2022', 'id': 1591536914808668160, 'id_str': '1591536914808668160', 'text': '5 most mentioned US stocks on Reddit in past 24 hours:\n1. $SPY : 27\n2. $NVDA : 15\n3. $TSLA : 13\n4. $META : 13\n5.… https://t.co/r0pdlwqb5u', 'truncated': True, 'entities': {'hashtags': [], 'symbols': [{'text': 'SPY', 'indices': [58, 62]}, {'text': 'NVDA', 'indices': [71, 76]}, {'text': 'TSLA', 'indices': [85, 90]}, {'text': 'META', 'indices': [99, 104]}], 'user_mentions': [], 'urls': [{'url': 'https://t.co/r0pdlwqb5u', 'expanded_url': 'https://twitter.com/i/web/status/1591536914808668160', 'display_url': 'twitter.com/i/web/status/1…', 'indices': [114, 137]}]}, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="https://help.twitter.com/en/using-twitter/how-to-tweet#source-labels" rel="nofollow">Reddit Stock Mention Bot</a>', 'in_reply_to_status_id': None, 'in_reply_to_

Status(_api=<tweepy.api.API object at 0x000002AF01C19D88>, _json={'created_at': 'Sat Nov 12 21:00:05 +0000 2022', 'id': 1591536334006624258, 'id_str': '1591536334006624258', 'text': 'Interested in the #stockmarket but not sure how to start trading? Why not become a #copytrader? - Learn more at… https://t.co/Ue1ZSokbix', 'truncated': True, 'entities': {'hashtags': [{'text': 'stockmarket', 'indices': [18, 30]}, {'text': 'copytrader', 'indices': [83, 94]}], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/Ue1ZSokbix', 'expanded_url': 'https://twitter.com/i/web/status/1591536334006624258', 'display_url': 'twitter.com/i/web/status/1…', 'indices': [113, 136]}]}, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="https://ifttt.com" rel="nofollow">IFTTT</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 123875775

Status(_api=<tweepy.api.API object at 0x000002AF01C19D88>, _json={'created_at': 'Sat Nov 12 21:02:14 +0000 2022', 'id': 1591536871242620928, 'id_str': '1591536871242620928', 'text': 'RT @MichaelMOTTCM: My latest video: Falling Liquidity Levels May Sink Stocks Into Year End https://t.co/bKTeAoBr4M #stocks #stockmarket #ma…', 'truncated': False, 'entities': {'hashtags': [{'text': 'stocks', 'indices': [115, 122]}, {'text': 'stockmarket', 'indices': [123, 135]}], 'symbols': [], 'user_mentions': [{'screen_name': 'MichaelMOTTCM', 'name': 'Michael J. Kramer', 'id': 700857611810422784, 'id_str': '700857611810422784', 'indices': [3, 17]}], 'urls': [{'url': 'https://t.co/bKTeAoBr4M', 'expanded_url': 'https://youtu.be/rbkitYr8Up0', 'display_url': 'youtu.be/rbkitYr8Up0', 'indices': [91, 114]}]}, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'in_reply_to_status_id': None, 'in_reply_to

Status(_api=<tweepy.api.API object at 0x000002AF01C19D88>, _json={'created_at': 'Sat Nov 12 21:02:26 +0000 2022', 'id': 1591536922190843904, 'id_str': '1591536922190843904', 'text': 'HIRING: Software Security Analyst / Seattle, WA, United States https://t.co/H4VW85uOQF #InfoSec #InfoSecJobs… https://t.co/76rBCAqGgI', 'truncated': True, 'entities': {'hashtags': [{'text': 'InfoSec', 'indices': [87, 95]}, {'text': 'InfoSecJobs', 'indices': [96, 108]}], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/H4VW85uOQF', 'expanded_url': 'https://infosec-jobs.com/J18980/', 'display_url': 'infosec-jobs.com/J18980/', 'indices': [63, 86]}, {'url': 'https://t.co/76rBCAqGgI', 'expanded_url': 'https://twitter.com/i/web/status/1591536922190843904', 'display_url': 'twitter.com/i/web/status/1…', 'indices': [110, 133]}]}, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="https://infosec-jobs.com" rel="nofollow">infosec-jobs.com</a>', 'in_reply_to_status_

Status(_api=<tweepy.api.API object at 0x000002AF01C19D88>, _json={'created_at': 'Sat Nov 12 20:50:14 +0000 2022', 'id': 1591533851507306497, 'id_str': '1591533851507306497', 'text': 'RT @SeekingAlpha: $PYPD - PolyPid Ltd. (PYPD) Q3 2022 Earnings Call Transcript. https://t.co/YGuit7r05v #business #finance #investing', 'truncated': False, 'entities': {'hashtags': [{'text': 'business', 'indices': [104, 113]}, {'text': 'finance', 'indices': [114, 122]}, {'text': 'investing', 'indices': [123, 133]}], 'symbols': [{'text': 'PYPD', 'indices': [18, 23]}], 'user_mentions': [{'screen_name': 'SeekingAlpha', 'name': 'Seeking Alpha', 'id': 23059499, 'id_str': '23059499', 'indices': [3, 16]}], 'urls': [{'url': 'https://t.co/YGuit7r05v', 'expanded_url': 'https://seekingalpha.com/article/4556947-polypid-ltd-pypd-q3-2022-earnings-call-transcript?source=feed_f&utm_campaign=twitter_automated&utm_content=article&utm_medium=social&utm_source=twitter_automated', 'display_url': 'seekingalpha.com/article/45569

...

Status(_api=<tweepy.api.API object at 0x000002AF01C19D88>, _json={'created_at': 'Sat Nov 12 09:18:12 +0000 2022', 'id': 1591359698283814912, 'id_str': '1591359698283814912', 'text': '$AMZN one of the most mentioned on wallstreetbets over the last 7 days\r\rVia https://t.co/pCi40pQv5N\r\r#amzn… https://t.co/DabWMPAXns', 'truncated': True, 'entities': {'hashtags': [{'text': 'amzn', 'indices': [101, 106]}], 'symbols': [{'text': 'AMZN', 'indices': [0, 5]}], 'user_mentions': [], 'urls': [{'url': 'https://t.co/pCi40pQv5N', 'expanded_url': 'http://topstonks.com/stocks/AMZN?st_amzn', 'display_url': 'topstonks.com/stocks/AMZN?st…', 'indices': [76, 99]}, {'url': 'https://t.co/DabWMPAXns', 'expanded_url': 'https://twitter.com/i/web/status/1591359698283814912', 'display_url': 'twitter.com/i/web/status/1…', 'indices': [108, 131]}]}, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="https://help.twitter.com/en/using-twitter/how-to-tweet#source-labels" rel="nofollo

Status(_api=<tweepy.api.API object at 0x000002AF01C19D88>, _json={'created_at': 'Sat Nov 12 02:53:03 +0000 2022', 'id': 1591262769688743936, 'id_str': '1591262769688743936', 'text': 'Investment portfolio \U0001f90d\U0001f90d\n\nCheck out my latest product: https://t.co/tdR05xWzAN\n\n#AAPL #StockMarkets #amzn #amc #MSFT #YaliCapkini #股票', 'truncated': False, 'entities': {'hashtags': [{'text': 'AAPL', 'indices': [79, 84]}, {'text': 'StockMarkets', 'indices': [85, 98]}, {'text': 'amzn', 'indices': [99, 104]}, {'text': 'amc', 'indices': [105, 109]}, {'text': 'MSFT', 'indices': [110, 115]}, {'text': 'YaliCapkini', 'indices': [116, 128]}, {'text': '股票', 'indices': [129, 132]}], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/tdR05xWzAN', 'expanded_url': 'https://payhip.com/b/zSvW9', 'display_url': 'payhip.com/b/zSvW9', 'indices': [54, 77]}]}, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="http://twitter.com/download/iphone" rel="nofol

Status(_api=<tweepy.api.API object at 0x000002AF01C19D88>, _json={'created_at': 'Sat Nov 12 02:30:14 +0000 2022', 'id': 1591257030429274112, 'id_str': '1591257030429274112', 'text': '#Amazon #AMZN #TechnicalIndicator Market Types (MT) \n-0.1658 | Fear \n\nView interactive details:… https://t.co/Oiv5yemVgI', 'truncated': True, 'entities': {'hashtags': [{'text': 'Amazon', 'indices': [0, 7]}, {'text': 'AMZN', 'indices': [8, 13]}, {'text': 'TechnicalIndicator', 'indices': [14, 33]}], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/Oiv5yemVgI', 'expanded_url': 'https://twitter.com/i/web/status/1591257030429274112', 'display_url': 'twitter.com/i/web/status/1…', 'indices': [97, 120]}]}, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="https://iobots.pro" rel="nofollow">ioBots Alerts</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_nam

Status(_api=<tweepy.api.API object at 0x000002AF01C19D88>, _json={'created_at': 'Fri Nov 11 21:17:36 +0000 2022', 'id': 1591178350470787073, 'id_str': '1591178350470787073', 'text': '@amzn #amzn $qqq #QQQ \n$100 came faster than I thought!  \n\n1. Heavy resistance $100-$105.  \n - Clear $105 and $110… https://t.co/jLtT9Jffqq', 'truncated': True, 'entities': {'hashtags': [{'text': 'amzn', 'indices': [6, 11]}, {'text': 'QQQ', 'indices': [17, 21]}], 'symbols': [{'text': 'qqq', 'indices': [12, 16]}], 'user_mentions': [{'screen_name': 'AMZN', 'name': 'AMZN Stock Quote', 'id': 2084301, 'id_str': '2084301', 'indices': [0, 5]}], 'urls': [{'url': 'https://t.co/jLtT9Jffqq', 'expanded_url': 'https://twitter.com/i/web/status/1591178350470787073', 'display_url': 'twitter.com/i/web/status/1…', 'indices': [116, 139]}]}, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>', 'in_reply_to_status_id': 1590731

Status(_api=<tweepy.api.API object at 0x000002AF01C19D88>, _json={'created_at': 'Fri Nov 11 18:26:34 +0000 2022', 'id': 1591135310485852180, 'id_str': '1591135310485852180', 'text': '#AMZN ABOVE ASK SWEEPER\n\n&gt;&gt;7028 AMZN Nov22 107 Calls $0.693  Above Ask! 13:24:56.240   AMZN = 100.86 Ref $487k… https://t.co/6uJ02gnqpJ', 'truncated': True, 'entities': {'hashtags': [{'text': 'AMZN', 'indices': [0, 5]}], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/6uJ02gnqpJ', 'expanded_url': 'https://twitter.com/i/web/status/1591135310485852180', 'display_url': 'twitter.com/i/web/status/1…', 'indices': [118, 141]}]}, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1231283070449438720, 'id_str': '12312

## Creating DataFrames based on the Twitter data related to the keywords

In [8]:
tweetdf = pd.DataFrame(outtweets, columns = ['Tweet ID', 'Screen Name', 'Name', 'Profile Image Url',  'Description', 'Followers Count', 'Following Count', 'User Creation Date', 'Tweet', 'Tags', 'Time', 'Mentions', 'Retweet Count'])
tbl_users_df = pd.DataFrame(tbl_users, columns = ['Screen Name', 'Name', 'Profile Image Url',  'Description', 'Followers Count', 'Following Count', 'User Creation Date'])
tbl_tweet_df = pd.DataFrame(tbl_tweet, columns = ['Tweet ID', 'Screen Name', 'Tweet', 'Profile Image Url', 'Time', 'Retweet Count'])
tbl_tags_df = pd.DataFrame(tbl_tags, columns = ['Tweet ID', 'Tag'])
tbl_mention_df = pd.DataFrame(tbl_mention, columns = ['Tweet ID', 'Source User', 'Target User'])

## Inserting Data into the MySQL Database 

In [9]:
#Iterate into the Master dataframe
for i,row in tweetdf.iterrows():
    #INSERT query for inserting the data into the database
    master_sql = "INSERT INTO stock_assist.Master_Twitter_Directory VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
    try:
        #Execute the SQL query
        mycursor.execute(master_sql, tuple(row))
        print('Master Directory Record inserted')
        #Commit the executed query
        mydb.commit()
    except:
        #Display message if the record already exists inside the table
        print('Master Directory Record already exists')


for i,row in tbl_users_df.iterrows():
    #INSERT query for inserting the data into the database
    tbl_users_sql = "INSERT INTO stock_assist.User VALUES (%s,%s,%s,%s,%s,%s,%s)"
    try:
        #Execute the SQL query
        mycursor.execute(tbl_users_sql, tuple(row))
        print('User Record inserted')
        #Commit the executed query
        mydb.commit()
    except:
        #Display message if the record already exists inside the table
        print('User Record already exists')

for i,row in tbl_tweet_df.iterrows():
    #INSERT query for inserting the data into the database
    tbl_tweet_sql = "INSERT INTO stock_assist.Tweets VALUES (%s,%s,%s,%s,%s,%s)"
    try:
        #Execute the SQL query
        mycursor.execute(tbl_tweet_sql, tuple(row))
        print('Tweet Record inserted')
        #Commit the executed query
        mydb.commit()
    except:
        #Display message if the record already exists inside the table
        print('Tweet Record already exists')


for i,row in tbl_tags_df.iterrows():
    #INSERT query for inserting the data into the database
    tbl_tags_sql = "INSERT INTO stock_assist.tweet_tags (twt_tweet_id, twt_tags) values (%s, %s)"
    try:
        #Execute the SQL query
        mycursor.execute(tbl_tags_sql, tuple(row))
        print('Tweet Tag Record inserted')
        #Commit the executed query
        mydb.commit()
    except:
        #Display message if the record already exists inside the table
        print('Tweet Tag Record already exists')


for i,row in tbl_mention_df.iterrows():
    #INSERT query for inserting the data into the database
    tbl_mention_sql = "INSERT INTO stock_assist.tweet_mentions (twm_tweet_id, twm_source_user, twm_target_user) values (%s, %s, %s)"
    try:
        #Execute the SQL query
        mycursor.execute(tbl_mention_sql, tuple(row))
        print('Tweet Mention Record inserted')
        #Commit the executed query
        mydb.commit()
    except:
        #Display message if the record already exists inside the table
        print('Tweet Mention Record already exists')


Master Directory Record inserted
Master Directory Record inserted
Master Directory Record inserted
Master Directory Record inserted
Master Directory Record inserted
Master Directory Record inserted
Master Directory Record inserted
Master Directory Record inserted
Master Directory Record inserted
Master Directory Record inserted
Master Directory Record inserted
User Record inserted
User Record inserted
User Record inserted
User Record inserted
User Record already exists
User Record already exists
User Record inserted
User Record already exists
User Record already exists
User Record already exists
User Record already exists
Tweet Record inserted
Tweet Record inserted
Tweet Record inserted
Tweet Record inserted
Tweet Record already exists
Tweet Record already exists
Tweet Record already exists
Tweet Record already exists
Tweet Record already exists
Tweet Record already exists
Tweet Record already exists
Tweet Tag Record inserted
Tweet Tag Record inserted
Tweet Tag Record inserted
Tweet Ta